In [3]:
from typing import Dict, List
from generator import GeneratorEngine
from generator import GeneratorFactory
from prompt_settings import AdvancedCfgNetPromptSettings
from dotenv import load_dotenv
from data import Dependency
from util import get_projet_description, get_most_similar_shots, load_shots
from collections import Counter
import pandas as pd
import backoff
import json
from tqdm import tqdm


def transform(row: pd.Series) -> Dependency:
    dependency = Dependency(
        project=row["project"],
        option_name=row["option_name"],
        option_value=row["option_value"],
        option_type=row["option_type"].split(".")[-1],
        option_file=row["option_file"],
        option_technology=row["option_technology"],
        dependent_option_name=row["dependent_option_name"],
        dependent_option_value=row["dependent_option_value"],
        dependent_option_type=row["dependent_option_type"].split(".")[-1],
        dependent_option_file=row["dependent_option_file"],
        dependent_option_technology=row["dependent_option_technology"]
    )

    return dependency


@backoff.on_exception(backoff.expo, Exception, max_tries=3)
def generate(generator: GeneratorEngine, messages: List) -> str:
    response = generator.generate(messages=messages)

    if not response:
        raise Exception("Response is empty.")
    
    try:
        response_dict = json.loads(response, strict=False)
        if "isDependency" not in response_dict:
            raise Exception("KeyError: isDependency")
    except json.JSONDecodeError:
        raise Exception("Response format not serializable.")

    return response


def run_generation(df: pd.DataFrame, model_name: str) -> List[str]:

    prompt_settings = AdvancedCfgNetPromptSettings
    generator = GeneratorFactory().get_generator(
        model_name=model_name, 
        temperature=0.0
    )
    shots = load_shots()

    results = []
    project_info = []
    shot_info = []



    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        dependency = transform(row=row)

        project_str = get_projet_description(project_name=row["project"])
        context_str = row["context_str"]
        task_str = prompt_settings.get_task_str(dependency=dependency)
        shots_str = "\n\n".join([shot for shot in get_most_similar_shots(shots, dependency)])
        format_str = prompt_settings.get_format_prompt()

        system_prompt = prompt_settings.get_system_str(
            dependency=dependency,
            project_str=project_str
        )

        user_prompt = prompt_settings.advanced_query_prompt.format(
                context_str=context_str, 
                shot_str=shots_str,
                task_str=task_str,
                format_str=format_str
        )

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        try:
            response = generate(
                generator=generator,
                messages=messages
            )
        except Exception:
            response = "None"

        shot_info.append(shots_str)
        project_info.append(project_str)
        results.append(response)

    df["responses"] = results
    df["shot_info"] = shot_info
    df["project_info"] = project_info

    df.to_csv(f"../data/analysis/failures_{model_name}.csv", index=False)

In [4]:
model_name = "gpt-4o-2024-05-13"
df_failures = pd.read_csv("../data/analysis/failures_annotated.csv")
df_failures = df_failures[df_failures["config"] == "config2"]
df_failures= df_failures[df_failures["llm"] == model_name]

print(f"Num Failures for {model_name}: {len(df_failures)}")

env_file = "../.env"
load_dotenv(dotenv_path=env_file)
run_generation(df=df_failures, model_name=model_name)

Num Failures for gpt-4o-2024-05-13: 116


[09/04/24 10:41:33] INFO     GPT (gpt-4o-2024-05-13) generator initialized.                         ]8;id=519663;file:///home/ssimon/projects/cval/src/generator.py\generator.py]8;;\:]8;id=450278;file:///home/ssimon/projects/cval/src/generator.py#88\88]8;;\

Processing rows:   0%|          | 0/116 [00:00<?, ?it/s]

[09/04/24 10:41:37] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=667068;file:///home/ssimon/projects/cval/cval_env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=447732;file:///home/ssimon/projects/cval/cval_env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

Processing rows:   1%|          | 1/116 [00:03<07:22,  3.85s/it]

...

Processing rows:  98%|█████████▊| 114/116 [07:51<00:08,  4.21s/it]

[09/04/24 10:49:28] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=46390;file:///home/ssimon/projects/cval/cval_env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=328073;file:///home/ssimon/projects/cval/cval_env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

Processing rows:  99%|█████████▉| 115/116 [07:55<00:04,  4.04s/it]

[09/04/24 10:49:31] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=767071;file:///home/ssimon/projects/cval/cval_env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=240911;file:///home/ssimon/projects/cval/cval_env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

Processing rows: 100%|██████████| 116/116 [07:58<00:00,  4.13s/it]
